# TSA Chapter 0: Time Series Decomposition

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_Ch0/TSA_ch0_decomposition/TSA_ch0_decomposition.ipynb)

This notebook demonstrates time series decomposition:
- Additive decomposition: $X_t = T_t + S_t + \varepsilon_t$
- Multiplicative decomposition: $X_t = T_t \times S_t \times \varepsilon_t$
- STL decomposition
- Cyclical component extraction

In [ ]:
!pip install matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.ndimage import uniform_filter1d
import statsmodels.api as sm
from statsmodels.tsa.seasonal import STL, seasonal_decompose

In [ ]:
# Style configuration
COLORS = {
    'blue': '#1A3A6E',
    'red': '#DC3545',
    'green': '#2E7D32',
    'orange': '#E67E22',
    'gray': '#666666',
    'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none',
    'figure.facecolor': 'none',
    'savefig.transparent': True,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 10,
    'axes.titlesize': 11,
    'axes.labelsize': 10,
    'legend.fontsize': 8,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'lines.linewidth': 1.5,
    'axes.prop_cycle': plt.cycler('color', list(COLORS.values())),
    'axes.edgecolor': '#333333',
    'axes.linewidth': 0.8,
    'axes.grid': False,
})

np.random.seed(42)

CHARTS_DIR = os.path.join(os.path.dirname(os.path.abspath('.')), '..', '..', 'charts')

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    # Also save to main charts directory for the lecture
    try:
        charts_path = os.path.join(CHARTS_DIR, name)
        fig.savefig(f'{charts_path}.pdf', bbox_inches='tight', transparent=True, dpi=150)
        fig.savefig(f'{charts_path}.png', bbox_inches='tight', transparent=True, dpi=150)
    except Exception:
        pass  # Skip if running on Colab without the charts dir
    print(f'Saved: {name}.pdf + .png')

def add_legend_below(ax, ncol=3):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=ncol, frameon=False)

In [ ]:
# Chart: ch1_decomposition
# Classical additive decomposition into 4 components
np.random.seed(42)
fig, axes = plt.subplots(4, 1, figsize=(8, 6), sharex=True)

t = np.arange(120)
trend = 100 + 0.5 * t
seasonal = 15 * np.sin(2 * np.pi * t / 12)
noise = np.random.randn(120) * 3
original = trend + seasonal + noise

axes[0].plot(t, original, color=COLORS['blue'], linewidth=1.2)
axes[0].set_title('Original Series', fontweight='bold', fontsize=10)
axes[0].set_ylabel('$X_t$')

axes[1].plot(t, trend, color=COLORS['red'], linewidth=1.5)
axes[1].set_title('Trend-Cycle', fontweight='bold', fontsize=10)
axes[1].set_ylabel('$T_t$')

axes[2].plot(t, seasonal, color=COLORS['green'], linewidth=1.2)
axes[2].set_title('Seasonal Component', fontweight='bold', fontsize=10)
axes[2].set_ylabel('$S_t$')

axes[3].plot(t, noise, color=COLORS['gray'], linewidth=0.8)
axes[3].set_title('Residuals', fontweight='bold', fontsize=10)
axes[3].set_ylabel('$\\varepsilon_t$')
axes[3].set_xlabel('Time (months)')

fig.tight_layout()
save_chart(fig, 'ch1_decomposition')
plt.show()

In [ ]:
# Chart: ch1_cyclical_component
# Extracting business cycle from economic data
np.random.seed(42)
fig, axes = plt.subplots(2, 1, figsize=(10, 4.5), sharex=True)

t = np.arange(200)
trend = 100 + 0.2 * t
cycle = 8 * np.sin(2 * np.pi * t / 60) + 4 * np.sin(2 * np.pi * t / 30)
seasonal = 5 * np.sin(2 * np.pi * t / 12)
original = trend + cycle + seasonal + np.random.randn(200) * 2

axes[0].plot(t, original, color=COLORS['blue'], linewidth=1.0, alpha=0.6, label='Original')
axes[0].plot(t, trend + cycle, color=COLORS['red'], linewidth=1.5, label='Trend + Cycle')
axes[0].set_title('Original Series with Trend-Cycle Component', fontweight='bold')
axes[0].set_ylabel('Value')
add_legend_below(axes[0], ncol=2)

axes[1].plot(t, cycle, color=COLORS['orange'], linewidth=1.5, label='Cyclic Component')
axes[1].axhline(y=0, color='black', linewidth=0.5)
axes[1].fill_between(t, 0, cycle, where=cycle > 0, alpha=0.15, color=COLORS['green'], label='Expansion')
axes[1].fill_between(t, 0, cycle, where=cycle < 0, alpha=0.15, color=COLORS['red'], label='Contraction')
axes[1].set_title('Isolated Cyclic Component', fontweight='bold')
axes[1].set_xlabel('Time (quarters)')
axes[1].set_ylabel('Cyclic Deviation')
add_legend_below(axes[1], ncol=3)

fig.tight_layout(rect=[0, 0.02, 1, 1])
save_chart(fig, 'ch1_cyclical_component')
plt.show()

In [ ]:
# Chart: ts_components_synthetic
# Four components displayed in 2x2 grid
np.random.seed(42)
fig, axes = plt.subplots(2, 2, figsize=(10, 5.5))

t = np.arange(120)
trend = 50 + 0.4 * t
seasonal = 12 * np.sin(2 * np.pi * t / 12)
noise = np.random.randn(120) * 2.5
original = trend + seasonal + noise

axes[0, 0].plot(t, trend, color=COLORS['red'], linewidth=1.5)
axes[0, 0].set_title('$T_t$: Trend', fontweight='bold')
axes[0, 0].set_ylabel('Value')

axes[0, 1].plot(t, seasonal, color=COLORS['green'], linewidth=1.2)
axes[0, 1].set_title('$S_t$: Seasonal Component', fontweight='bold')

axes[1, 0].plot(t, noise, color=COLORS['gray'], linewidth=0.8)
axes[1, 0].set_title('$\\varepsilon_t$: Residuals', fontweight='bold')
axes[1, 0].set_xlabel('Time')
axes[1, 0].set_ylabel('Value')

axes[1, 1].plot(t, original, color=COLORS['blue'], linewidth=1.2)
axes[1, 1].set_title('$X_t = T_t + S_t + \\varepsilon_t$: Original', fontweight='bold')
axes[1, 1].set_xlabel('Time')

fig.suptitle('Additive Decomposition', fontweight='bold', fontsize=12, y=1.02)
fig.tight_layout()
save_chart(fig, 'ts_components_synthetic')
plt.show()

In [ ]:
# Chart: airline_decomposition
# Multiplicative decomposition of real AirPassengers data
try:
    air_df = sm.datasets.get_rdataset('AirPassengers').data
    air_values = air_df['value'].values.astype(float)
    air_time = pd.date_range('1949-01', periods=len(air_values), freq='MS')
    print(f'AirPassengers loaded: {len(air_values)} observations')
except Exception:
    np.random.seed(42)
    t_ap = np.arange(144)
    trend_ap = 100 + 1.8 * t_ap + 0.005 * t_ap**2
    sf_ap = 1 + 0.15 * np.sin(2 * np.pi * t_ap / 12) + 0.08 * np.cos(4 * np.pi * t_ap / 12)
    air_values = trend_ap * sf_ap * (1 + np.random.randn(144) * 0.03)
    air_time = pd.date_range('1949-01', periods=144, freq='MS')
    print('Using synthetic fallback for AirPassengers')

air_series = pd.Series(air_values, index=air_time)
decomp = seasonal_decompose(air_series, model='multiplicative', period=12)

fig, axes = plt.subplots(4, 1, figsize=(8, 6), sharex=True)

axes[0].plot(air_time, air_values, color=COLORS['blue'], linewidth=1.2)
axes[0].set_title('Original Data (AirPassengers)', fontweight='bold', fontsize=10)
axes[0].set_ylabel('Passengers')

axes[1].plot(air_time, decomp.trend, color=COLORS['red'], linewidth=1.5)
axes[1].set_title('Trend', fontweight='bold', fontsize=10)
axes[1].set_ylabel('$T_t$')

axes[2].plot(air_time, decomp.seasonal, color=COLORS['green'], linewidth=1.2)
axes[2].axhline(y=1, color='black', linewidth=0.5, linestyle=':')
axes[2].set_title('Seasonal Factor (multiplicative)', fontweight='bold', fontsize=10)
axes[2].set_ylabel('$S_t$')

axes[3].plot(air_time, decomp.resid, color=COLORS['gray'], linewidth=0.8)
axes[3].axhline(y=1, color='black', linewidth=0.5, linestyle=':')
axes[3].set_title('Residuals', fontweight='bold', fontsize=10)
axes[3].set_ylabel('$\\varepsilon_t$')
axes[3].set_xlabel('Year')

fig.tight_layout()
save_chart(fig, 'airline_decomposition')
plt.show()

In [ ]:
# Chart: additive_vs_multiplicative
# Side-by-side comparison of additive and multiplicative decomposition
np.random.seed(42)
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

t = np.arange(120)

# Additive (synthetic - constant seasonal amplitude)
trend_a = 50 + 0.3 * t
seasonal_a = 10 * np.sin(2 * np.pi * t / 12)
y_add = trend_a + seasonal_a + np.random.randn(120) * 2
axes[0].plot(t, y_add, color=COLORS['blue'], linewidth=1.2, label='Data')
axes[0].plot(t, trend_a, color=COLORS['red'], linewidth=1.5, linestyle='--', label='Trend')
axes[0].set_title('Additive Decomposition\n$X_t = T_t + S_t + \\varepsilon_t$', fontweight='bold')
axes[0].set_xlabel('Time')
axes[0].set_ylabel('$X_t$')
axes[0].annotate('Constant\namplitude', xy=(60, trend_a[60]+10), fontsize=8,
                 color=COLORS['green'], fontweight='bold',
                 arrowprops=dict(arrowstyle='->', color=COLORS['green'], lw=1),
                 xytext=(75, trend_a[60]+20))
add_legend_below(axes[0], ncol=2)

# Multiplicative (real AirPassengers - increasing seasonal amplitude)
axes[1].plot(np.arange(len(air_values)), air_values, color=COLORS['blue'], linewidth=1.2, label='AirPassengers')
air_trend = uniform_filter1d(air_values.astype(float), size=12)
axes[1].plot(np.arange(len(air_values)), air_trend, color=COLORS['red'], linewidth=1.5, linestyle='--', label='Trend')
axes[1].set_title('Multiplicative Decomposition\n$X_t = T_t \\times S_t \\times \\varepsilon_t$', fontweight='bold')
axes[1].set_xlabel('Month (1949-1960)')
axes[1].set_ylabel('Passengers')
axes[1].annotate('Increasing\namplitude', xy=(120, air_values[120] if len(air_values) > 120 else air_values[-1]), fontsize=8,
                 color=COLORS['green'], fontweight='bold',
                 arrowprops=dict(arrowstyle='->', color=COLORS['green'], lw=1),
                 xytext=(130, air_values[60] if len(air_values) > 60 else air_values[0]))
add_legend_below(axes[1], ncol=2)

fig.tight_layout(rect=[0, 0.02, 1, 1])
save_chart(fig, 'additive_vs_multiplicative')
plt.show()

In [ ]:
# Chart: ch1_def_stl
# STL decomposition of real AirPassengers data
air_stl_series = pd.Series(air_values, index=air_time)
stl = STL(air_stl_series, period=12, robust=True)
stl_result = stl.fit()

fig, axes = plt.subplots(4, 1, figsize=(8, 6), sharex=True)

axes[0].plot(air_time, air_values, color=COLORS['blue'], linewidth=1.2)
axes[0].set_title('Original Series (AirPassengers)', fontweight='bold', fontsize=10)
axes[0].set_ylabel('$X_t$')

axes[1].plot(air_time, stl_result.trend, color=COLORS['red'], linewidth=1.5)
axes[1].set_title('Trend (LOESS)', fontweight='bold', fontsize=10)
axes[1].set_ylabel('$T_t$')

axes[2].plot(air_time, stl_result.seasonal, color=COLORS['green'], linewidth=1.2)
axes[2].set_title('Seasonal Component (time-varying)', fontweight='bold', fontsize=10)
axes[2].set_ylabel('$S_t$')

axes[3].plot(air_time, stl_result.resid, color=COLORS['gray'], linewidth=0.8)
axes[3].axhline(y=0, color='black', linewidth=0.5)
axes[3].set_title('Residuals', fontweight='bold', fontsize=10)
axes[3].set_xlabel('Year')
axes[3].set_ylabel('$R_t$')

fig.suptitle('STL Decomposition (Seasonal-Trend-Loess)', fontweight='bold', fontsize=12, y=1.02)
fig.tight_layout()
save_chart(fig, 'ch1_def_stl')
plt.show()